# Final Project

In this project we will have three topics

1) Asteroid Spectral Classification Estimator: 

The idea comes from [Science Projects: Asteroid Mining](https://www.sciencebuddies.org/science-fair-projects/project-ideas/Astro_p038/astronomy/asteroid-mining-gold-rush-in-space).

There are about 1.2 million asteroid data in the [NASA JPL Small-Body Database](https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/), however not every data has complete parameters.

We will use several orbital parameters, asteroid size, and bulk albedo to predict possible spectral classifications. It is actually a classification challenge, KNN is used here since the number of data points with known spectral type is less than 1000 in the JPL data set.

Then, I use MCMC to find the best parameters. Science Projects: Asteroid Mining

2) MCMC Mcfost:

My current research topic is exoplanetary system disk modeling, which could be essential to model the performance of coronagraphs, like Roman CGI.

In this chapter, we will introduce what Mcfost does, and how to modify the parameters with the para file. Then, we will use the MCMC method to figure out the best-fitting para file.

RZ Psc will be used as an example in the simulation, see [dreamjade/mcfost-python](https://github.com/dreamjade/mcfost-python) for detail.

3) Flavor transformation Collision-term:

In this chapter, we will discuss monochromatic Collision-induced flavor instability.

See [dreamjade/CollisionTerm](https://github.com/dreamjade/CollisionTerm) for detail.

4) Asteroid Mass Estimator (backup topic):

There has been little quantitative analysis of asteroid masses in the NASA JPL dataset,  the majority of the data points lack GM values. That is because an accurate asteroid masses estimation usually requires close encounters or high-accuracy orbit observation of a multiple-body system. However, mass is one of the most crucial asteroid features. One may determine the potential interior composition of an asteroid by combining its mass and size. The makeup of asteroids is a reflection of the accretion and collisional environment that existed in the early solar system. Therefore, it could be useful if we could predict the possible mass from the known parameters. In this part, we will build a neural network to give the possible mass range of asteroids with unknown mass. Similar concepts have been discussed in [Agnan 2021](https://www.sciencedirect.com/science/article/pii/S0273117721004622). 

However, considering that quality is affected by multiple observables, only 15 GM data points are likely to be insufficient for even the most preliminary analysis. As a result, we need to gather information from papers about asteroid masses. mass estimates could be done according to orbital deflections during close encounters ([Michalak 2000](https://articles.adsabs.harvard.edu/pdf/2000A%26A...360..363M), [2001](https://www.aanda.org/articles/aa/pdf/2001/29/aa10228.pdf); [Zielenbach 2011](https://iopscience.iop.org/article/10.1088/0004-6256/142/4/120/pdf)), planetary ephemerides ([Baer & Chesley 2008](https://link.springer.com/content/pdf/10.1007/s10569-007-9103-8.pdf); [Baer et al. 2011](https://iopscience.iop.org/article/10.1088/0004-6256/141/5/143/pdf); [Fienga et al. 2008](https://www.aanda.org/articles/aa/pdf/2008/49/aa6607-06.pdf), [2009](https://www.aanda.org/articles/aa/pdf/2009/45/aa11755-09.pdf), [2014](https://arxiv.org/pdf/1405.0484.pdf)), and a comprehensive review of prior research ([Density of asteroids by B. Carry](https://arxiv.org/pdf/1203.4336.pdf)). There are also some recently results based on the data from ESA Gaia mission ([Siltala & Granvik 2021A](https://www.aanda.org/articles/aa/full_html/2022/02/aa41459-21/aa41459-21.html),[2021B](https://iopscience.iop.org/article/10.3847/2041-8213/abe948)) .

Additionally, as the volume is the third power of the dimension, the inferred mass error could result from inaccurate dimensions, which would result in a threefold increase in the error ([Hanuš et al. 2017](https://www.aanda.org/component/article?access=doi&doi=10.1051/0004-6361/201629956#R32)).

5) Other Useful Links:

[List of exceptional asteroids on Wiki](https://en.wikipedia.org/wiki/List_of_exceptional_asteroids)

[Using Bayesian Deep Learning to Infer Planet Mass from Gaps in Protoplanetary Disks](https://iopscience.iop.org/article/10.3847/1538-4357/ac7a3c)

## 1 Asteroid Spectral Classification Estimator

### Read data

In [40]:
import csv
#opening the csv file by specifying
with open('sbdb_asteroids.csv') as csv_file:
    # Creating an object of csv reader
    csv_reader = csv.reader(csv_file, delimiter = ',')
    columns = []
 
    # loop to iterate through the rows of csv
    for row in csv_reader:
        # Write columns
        columns.append(row)
# printing the result
column_names = columns[0]
print("List of column names: ", columns[0])
print("Total data point number: "+ str(len(columns)))

List of column names:  ['spec_B', 'spec_T', 'full_name', 'diameter', 'extent', 'albedo', 'a', 'q', 'i', 'GM', 'rot_per', 'BV', 'UB', 'IR']
Total data point number: 1242581


In [886]:
for i in range(len(column_names)):
    non_empty_number = 0
    for data in columns[1:]:
        if data[i] !='':
            non_empty_number += 1
    print("The number of data with known "+column_names[i]+": "+ str(non_empty_number))

The number of data with known spec_B: 1666
The number of data with known spec_T: 980
The number of data with known full_name: 1242580
The number of data with known diameter: 139680
The number of data with known extent: 20
The number of data with known albedo: 138546
The number of data with known a: 1242580
The number of data with known q: 1242580
The number of data with known i: 1242580
The number of data with known GM: 15
The number of data with known rot_per: 33350
The number of data with known BV: 1021
The number of data with known UB: 979
The number of data with known IR: 1


In [887]:
# Spec_B list
spec_B_list = [i[0] for i in columns[1:]]

# Get the set of all possible elements in the list
all_elements = set(spec_B_list)

# Count the number of times each element appears
counts = {}
for elem in all_elements:
    counts[elem] = spec_B_list.count(elem)

# Print the counts
sorted_counts = dict(sorted(counts.items(), key=lambda x: x[1], reverse=True))
print(sorted_counts)

{'': 1240914, 'S': 445, 'C': 152, 'Ch': 139, 'X': 138, 'Sq': 114, 'Xc': 67, 'B': 66, 'Sl': 56, 'Xk': 48, 'V': 48, 'L': 41, 'Sa': 38, 'Cb': 37, 'K': 37, 'Xe': 30, 'Sk': 29, 'Sr': 27, 'Q': 20, 'T': 19, 'A': 17, 'S:': 16, 'Cgh': 15, 'Ld': 15, 'D': 13, 'Cg': 9, 'O': 7, 'X:': 6, 'R': 5, 'U': 4, 'C:': 3, 'Sq:': 2, 'V:': 1, 'K:': 1, 'S(IV)': 1}


### Asteroid spectral types (SMASS, "spec_B")

reference: [wiki/Asteroid_spectral_types](https://en.wikipedia.org/wiki/Asteroid_spectral_types)

SMASS("spec_B" based on spectral features from 0.44 μm to 0.92 μm) is chosen as the target since it doesn't base on the albedo and we have more data points with this value. Here, instead of using detailed classification, I adopted 3 big groups and other as spectral classification.

Below is the information for the 3 major groups:

1) C-group: 

Carbon-based objects, including 'C','Ch','B','Cb','Cgh','Cg','C:' in the data set.

2) S-group:

Silicate majored objects, including 'S','Sq','Sl','L','Sa','K','Sk','Sr','Q','A','S:','R','Sq:','K:','S(IV)' in the data set.

3) X-group: 

Mostly metallic objects, including 'X','Xc','Xk','Xe','X:' in the data set..

4) Other: 

Whose spectrum cannot classification into the previous three groups, including 'V','T','Ld','D','O','U','V:' in the data set.




### knn codes

In [852]:
import numpy as np
from sklearn.model_selection import KFold
from scipy.spatial import distance

# Define the k-NN algorithm
def knn(X, y, x, k, std_adj = np.array([0.51701419,0.20836071,0.32275631,0.40417267]), q=2.5, r= 2):
    # Normalize the data
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_std[1:] = X_std[1:]*std_adj # wighted by changing the std
    X_norm = (X - X_mean) / X_std
    x_norm = (x - X_mean) / X_std
    
    # Compute the Minkowski distances between x and all points in X
    distances = distance.cdist (x_norm.reshape(1, -1), X_norm, 'minkowski', p=q)[0]
    
    # Sort the distances and the corresponding labels
    sorted_indices = np.argsort(distances)
    sorted_distances = distances[sorted_indices]
    sorted_labels = y[sorted_indices]
    
    #Check if inside the data points
    if not sorted_distances[0]:
        #print("in train data")
        return sorted_labels[0]
    
    # Take the k-nearest neighbors
    nearest_neighbors = sorted_labels[:k]

    # Compute the weights based on the distances
    weights = 1 / sorted_distances[:k]**r

    # Normalize the weights
    weights /= np.sum(weights)

    # Compute the weighted average of the labels
    predicted_label = np.dot(nearest_neighbors.T, weights)
    
    # Maximize
    final_predicted = np.zeros(len(predicted_label))
    final_predicted[np.argmax(predicted_label)] = 1
    
    return final_predicted

def knn_mul(X, y, x, k, std_adj = np.array([1,1,1,1]), q=2, r= 2):
    # output
    final_predicted = np.zeros((x.shape[0], y.shape[1]))
    
    # Normalize the data
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X_std[1:] = X_std[1:]*std_adj # wighted by changing the std
    X_norm = (X - X_mean) / X_std
    x_norm = (x - X_mean) / X_std
    
    # Compute the Minkowski distances between x and all points in X
    distances = distance.cdist (x_norm, X_norm, 'minkowski', p=q)
    
    # Sort the distances and the corresponding labels
    sorted_indices = np.argsort(distances)
    sorted_distances = distances[np.arange(distances.shape[0])[:, None], sorted_indices]
    ys = np.tile(y, (len(x),1,1))
    sorted_labels = ys[np.arange(ys.shape[0])[:, None], sorted_indices]
    # Take the k-nearest neighbors
    nearest_neighbors = sorted_labels[:,:k]

    # Compute the weights based on the distances
    nearest_distances = sorted_distances[:,:k]
    weights = (np.linalg.norm(nearest_distances, ord=-r, axis=-1)[:, np.newaxis] / nearest_distances)**r

    # Compute the weighted average of the labels
    for i in range(x.shape[0]):
        predicted_label = np.dot(nearest_neighbors[i].T, weights[i])
        # Maximize
        final_predicted[i, np.argmax(predicted_label)] = 1
    
    return final_predicted

#### Before mcmc, using normalization and Euclidean distance

In [890]:
# Spec_B list
paraB = [[i[3], i[5], i[6], i[7], i[8]] for i in columns[1:] if '' not in [i[0], i[3], i[5], i[6], i[7], i[8]]]
dataB = [i[0] for i in columns[1:] if '' not in [i[0], i[3], i[5], i[6], i[7], i[8]]]

#[C,S,X,O]
dataB_number_ratio = [1, 1, 1]
dataB_number = [[1,0,0,0] if i in ['C','Ch','B','Cb','Cgh','Cg','C:'] else i for i in dataB]
dataB_number = [[0,dataB_number_ratio[0],0,0] if i in ['S','Sq','Sl','L','Sa','K','Sk','Sr','Q','A','S:','R','Sq:','K:','S(IV)'] else i for i in dataB_number]
dataB_number = [[0,0,dataB_number_ratio[1],0] if i in ['X','Xc','Xk','Xe','X:'] else i for i in dataB_number]
dataB_number = [[0,0,0,dataB_number_ratio[2]] if i in ['V','T','Ld','D','O','U','V:'] else i for i in dataB_number]

# Define the training data
X = np.array([[float(j) for j in i] for i in paraB])
y = np.array(dataB_number)

# print prob
def print_prob(prob_list):
    print('C: '+'{:.2%}'.format(prob_list[0]))
    print('S: '+'{:.2%}'.format(prob_list[1]))
    print('X: '+'{:.2%}'.format(prob_list[2]))
    print('O: '+'{:.2%}'.format(prob_list[3]))

# Define the test point
x = [[i[3], i[5], i[6], i[7], i[8]] for i in columns[1:] if '' not in [i[3], i[5], i[6], i[7], i[8]]]
x = np.array([[float(j) for j in i] for i in x])
#predicted_label = knn(X, y, x[15600], k=5)
#print_prob(predicted_label)

# print accuaracy table
def accu_tab(fact, predict):
    if fact.shape!=predict.shape:
        return 0
    table_size = len(predict[0])
    accu_tab = np.zeros([table_size,table_size])
    for i in range(len(predict)):
        accu_tab = accu_tab + np.outer(fact[i], predict[i])
    return accu_tab/len(predict)

# Run cross-validation to evaluate the performance of the k-NN algorithm
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
scores = []
accu_tabs = []
for train_index, test_index in kfold.split(X):
    #print(len(train_index), len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_test = np.array([[1 if j !=0 else 0 for j in i] for i in y_test]) # normalize y_test
    predicted_labels = knn_mul(X_train, y_train, X_test, 23, np.array([1,1,1,1]), 2, 1)
    score = np.sum(np.apply_along_axis(np.average,1,(y_test*predicted_labels).T))
    # Print the accuaracy matrix as percentages
    accu_tabs.append(accu_tab(y_test, predicted_labels))
    scores.append(score)

# Print the mean accuracy of the k-NN algorithm
print('Mean Accuracy: '+'{:.3%}'.format(np.mean(scores)))
# Configure the NumPy printing options
#np.set_printoptions(formatter={'float': '{: 0.3%}'.format})
#np.set_printoptions(threshold=1000, edgeitems=3, linewidth=75, precision=8, suppress=False, nanstr='nan', infstr='inf', formatter=None)

Mean Accuracy: 72.062%


#### After mcmc

In [884]:
# Spec_B list
paraB = [[i[3], i[5], i[6], i[7], i[8]] for i in columns[1:] if '' not in [i[0], i[3], i[5], i[6], i[7], i[8]]]
dataB = [i[0] for i in columns[1:] if '' not in [i[0], i[3], i[5], i[6], i[7], i[8]]]

#[C,S,X,O]
dataB_number_ratio = [0.80611106,   0.85283428,   0.63251724]
dataB_number = [[1,0,0,0] if i in ['C','Ch','B','Cb','Cgh','Cg','C:'] else i for i in dataB]
dataB_number = [[0,dataB_number_ratio[0],0,0] if i in ['S','Sq','Sl','L','Sa','K','Sk','Sr','Q','A','S:','R','Sq:','K:','S(IV)'] else i for i in dataB_number]
dataB_number = [[0,0,dataB_number_ratio[1],0] if i in ['X','Xc','Xk','Xe','X:'] else i for i in dataB_number]
dataB_number = [[0,0,0,dataB_number_ratio[2]] if i in ['V','T','Ld','D','O','U','V:'] else i for i in dataB_number]

# Define the training data
X = np.array([[float(j) for j in i] for i in paraB])
y = np.array(dataB_number)

# print prob
def print_prob(prob_list):
    print('C: '+'{:.2%}'.format(prob_list[0]))
    print('S: '+'{:.2%}'.format(prob_list[1]))
    print('X: '+'{:.2%}'.format(prob_list[2]))
    print('O: '+'{:.2%}'.format(prob_list[3]))

# Define the test point
x = [[i[3], i[5], i[6], i[7], i[8]] for i in columns[1:] if '' not in [i[3], i[5], i[6], i[7], i[8]]]
x = np.array([[float(j) for j in i] for i in x])
#predicted_label = knn(X, y, x[15600], k=5)
#print_prob(predicted_label)

# print accuaracy table
def accu_tab(fact, predict):
    if fact.shape!=predict.shape:
        return 0
    table_size = len(predict[0])
    accu_tab = np.zeros([table_size,table_size])
    for i in range(len(predict)):
        accu_tab = accu_tab + np.outer(fact[i], predict[i])
    return accu_tab/len(predict)

# Run cross-validation to evaluate the performance of the k-NN algorithm
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
scores = []
accu_tabs = []
for train_index, test_index in kfold.split(X):
    #print(len(train_index), len(test_index))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_test = np.array([[1 if j !=0 else 0 for j in i] for i in y_test]) # normalize y_test
    predicted_labels = knn_mul(X_train, y_train, X_test, 23, np.array([0.93,1.10,1.19,1.53]), 0.96, 1.38)
    score = np.sum(np.apply_along_axis(np.average,1,(y_test*predicted_labels).T))
    # Print the accuaracy matrix as percentages
    accu_tabs.append(accu_tab(y_test, predicted_labels))
    scores.append(score)

# Print the mean accuracy of the k-NN algorithm
print('Mean Accuracy: '+'{:.3%}'.format(np.mean(scores)))
# Configure the NumPy printing options
#np.set_printoptions(formatter={'float': '{: 0.3%}'.format})
#np.set_printoptions(threshold=1000, edgeitems=3, linewidth=75, precision=8, suppress=False, nanstr='nan', infstr='inf', formatter=None)

Mean Accuracy: 74.415%


In [885]:
# Python program to understand the usage of tabulate function for printing tables in a tabular format
from tabulate import tabulate
titles = ["C", "S", "X", "O"]
accu_tabs_mean=np.mean(accu_tabs, axis=0)
accu_tabs_p = []
for i in range(len(titles)):
    accu_tabs_p.append([titles[i]]+[format(x, '.2%') for x in accu_tabs_mean[i]])
print (tabulate(accu_tabs_p, headers=["O\E","C", "S", "X", "O"]))
#print(np.trace(np.mean(accu_tabs, axis=0)))

O\E    C       S       X      O
-----  ------  ------  -----  -----
C      26.59%  1.57%   0.64%  0.00%
S      1.85%   44.47%  0.93%  0.07%
X      8.91%   6.20%   2.85%  0.07%
O      1.71%   3.20%   0.43%  0.50%


In [891]:
# type * happens prob
predicted_labels = knn_mul(X_train, y_train, x, 25, np.array([0.88,1.2,0.68,0.95]), 0.7, 1.5)
print_prob(np.apply_along_axis(np.average,1,np.array(predicted_labels).T))

/tmp/ipykernel_980889/3851234169.py:69: RuntimeWarning: invalid value encountered in divide
  weights = (np.linalg.norm(nearest_distances, ord=-r, axis=-1)[:, np.newaxis] / nearest_distances)**r


C: 60.29%
S: 35.71%
X: 3.22%
O: 0.77%


#### MCMC part

In [854]:
#record best
best_sol_dir='knn_best.npy'
#np.save(best_sol_dir, np.array([-31.97546022138345,0.5,0.5,0.5,0.5,1,1,1,39,2,2]))

In [855]:
import emcee
def lnlike(theta):
    std_adj = theta[:4]
    number_mul = theta[4:7]
    k = int(theta[7])
    q = theta[8]
    r = theta[9]
    #[C,S,X,O]
    dataB_number = [[1,0,0,0] if i in ['C','Ch','B','Cb','Cgh','Cg','C:'] else i for i in dataB]
    dataB_number = [[0,number_mul[0],0,0] if i in ['S','Sq','Sl','L','Sa','K','Sk','Sr','Q','A','S:','R','Sq:','K:','S(IV)'] else i for i in dataB_number]
    dataB_number = [[0,0,number_mul[1],0] if i in ['X','Xc','Xk','Xe','X:'] else i for i in dataB_number]
    dataB_number = [[0,0,0,number_mul[2]] if i in ['V','T','Ld','D','O','U','V:'] else i for i in dataB_number]

    # Define the training data
    X = np.array([[float(j) for j in i] for i in paraB])
    y = np.array(dataB_number)

    # Run cross-validation to evaluate the performance of the k-NN algorithm
    kfold = KFold(n_splits=10, shuffle=True, random_state=1)
    scores = []
    for train_index, test_index in kfold.split(X):
        #print(len(train_index), len(test_index))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_test = np.array([[1 if j !=0 else 0 for j in i] for i in y_test]) # normalize y_test
        predicted_labels = knn_mul(X_train, y_train, X_test, k, std_adj,q,r)
        score = np.sum(np.apply_along_axis(np.average,1,(y_test*predicted_labels).T))
        scores.append(score)
    output = 100*np.log(np.mean(scores))
    global best_sol_dir
    try:best_sol = np.load(best_sol_dir,)
    except:return(output)
    if output>best_sol[0]:
        best_sol[0]=output
        best_sol[1:]=theta
        print(best_sol)
        np.save(best_sol_dir, best_sol)
    return(output)

In [856]:
def lnprior(theta):
    '''
    if theta[7]<1 or theta[7]>100:
        return -np.inf
    for i in theta[:7]:
        if i<0.2 or i>5:
            return -np.inf
    for i in theta[8:10]:
        if i<0.2 or i>5:
            return -np.inf
    '''
    global pos0
    if distance.minkowski(np.delete(theta, -3), np.delete(pos0, -3), 1)>0.5:
        return -np.inf
    if theta[7]<1 or theta[7]>100:
        return -np.inf
    return 0

def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta)

In [857]:
#random position
pos0 = np.array([0.91158577,  1.03691   ,  1.18240536,  1.5094504 ,  0.76005638,
        0.90478132,  0.6032742 , 23.56535226,  0.97658515,  1.40117865])
pos = np.random.rand(1024, 10)*[0.05,0.05,0.05,0.05,0.05,0.05,0.05,30,0.05,0.05]+pos0
#pos = np.random.rand(1024, 10)*[4.8,4.8,4.8,4.8,4.8,4.8,4.8,99,4.8,4.8]+np.ones((1024, 10))*[0.2,0.2,0.2,0.2,0.2,0.2,0.2,1,0.2,0.2]
#pos[0] = np.array([0.5,0.5,0.5,0.5,1,1,1,39,2,2])
nwalkers, ndim = pos.shape
filename = "Best_fit_knn.h5"
knn_backend = emcee.backends.HDFBackend(filename, name="Best_fit_knn")
# Don't forget to clear it in case the file already exists
#mcfost_backend.reset(nwalkers, ndim)

In [ ]:
#run mcmc
from multiprocessing import Pool
import os
os.environ["OMP_NUM_THREADS"] = "16"
steps = 6000
with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, pool=pool, backend=knn_backend)
    sampler.run_mcmc(pos, steps, progress=True)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(10, figsize=(10, 30), sharex=True)
samples = sampler.get_chain()
labels = ['std_adj','std_adj','std_adj','std_adj','number_mul','number_mul','number_mul','k','q','r']
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
axes[-1].set_xlabel("step number")

In [ ]:
sampler.get_autocorr_time(tol=0)

sampler.get_autocorr_time(tol=0)

100%|██████████| 16000/16000 [27:00:51<00:00,  6.08s/it]

Detailed performance metrics for this job will be available at https://metrics.hpc.arizona.edu/#job_viewer?action=show&realm=SUPREMM&resource_id=73&local_job_id=5794790 by 8am on 2022/12/08.